In [1]:
from risk_targeted_hazard import *  # note yet a pypi package, must be installed locally

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
hazard_id = 'SLT_v8_gmm_v2_FINAL'

res = next(get_hazard_curves(['-36.870~174.770'], [400], [hazard_id], ['PGA'], ['mean']))
num_levels = len(res.values)

### query data for a short list of sites

In [3]:
if True:
    custom_location_dict = LOCATIONS_BY_ID.copy()
    del custom_location_dict['WRE']
    locations = [f"{loc['latitude']:0.3f}~{loc['longitude']:0.3f}" for loc in custom_location_dict.values()]

    vs30_list = [150,175,200,225,250,275,300,350,375,400,450,500,600,750,900,1000,1500]
    imts = ['PGA', 'SA(0.1)', 'SA(0.2)', 'SA(0.3)', 'SA(0.4)', 'SA(0.5)', 'SA(0.7)','SA(1.0)', 'SA(1.5)', 'SA(2.0)', 'SA(3.0)', 'SA(4.0)', 'SA(5.0)', 'SA(6.0)','SA(7.5)', 'SA(10.0)']
    # aggs = ["mean", "cov", "std", "0.005", "0.01", "0.025", "0.05", "0.1", "0.2", "0.5", "0.8", "0.9", "0.95", "0.975", "0.99", "0.995"]
    aggs = ["mean","0.1","0.5","0.9"]
    
else:
    # shorter list for testing
    site_codes = ['AKL','WLG']
    custom_location_dict = LOCATIONS_BY_ID.copy()
    new_custom_location_dict = {}
    for site_code in site_codes:
        new_custom_location_dict[site_code] = custom_location_dict[site_code]
    custom_location_dict = new_custom_location_dict
    locations = [f"{loc['latitude']:0.3f}~{loc['longitude']:0.3f}" for loc in custom_location_dict.values()]

    vs30_list = [250,400]
    imts = ['SA(0.5)', 'SA(1.5)']
    # aggs = ["mean", "cov", "std", "0.005", "0.01", "0.025", "0.05", "0.1", "0.2", "0.5", "0.8", "0.9", "0.95", "0.975", "0.99", "0.995"]
    aggs = ["mean","0.1","0.5","0.9"]


In [4]:
columns = ['lat', 'lon', 'vs30','imt', 'agg', 'level', 'hazard']
index = range(len(locations) * len(vs30_list) * len(imts) * len(aggs) * num_levels)
pts_summary_data = pd.DataFrame(columns=columns, index=index)

ind = 0
for i,res in enumerate(get_hazard_curves(locations, vs30_list, [hazard_id], imts, aggs)):
    lat = f'{res.lat:0.3f}'
    lon = f'{res.lon:0.3f}'
    for value in res.values:
        pts_summary_data.loc[ind,'lat'] = lat
        pts_summary_data.loc[ind,'lon'] = lon
        pts_summary_data.loc[ind,'vs30'] = res.vs30
        pts_summary_data.loc[ind,'imt'] = res.imt
        pts_summary_data.loc[ind,'agg'] = res.agg
        pts_summary_data.loc[ind,'level'] = value.lvl
        pts_summary_data.loc[ind,'hazard'] = value.val
        ind += 1
        

In [5]:
site_list = sorted([loc['name'] for loc in custom_location_dict.values()])
sites = pd.DataFrame(index=site_list,dtype='str')
for loc in custom_location_dict.values():
    idx = (pts_summary_data['lat'].astype('float')==loc['latitude'])&(pts_summary_data['lon'].astype('float')==loc['longitude'])
    pts_summary_data.loc[idx,'name'] = loc['name']
    sites.loc[loc['name'],['lat','lon']] = [loc['latitude'],loc['longitude']]


### query data for a gridded map

In [ ]:
vs30_list = [250,400]
imts = ['PGA', 'SA(0.5)', 'SA(1.0)', 'SA(1.5)', 'SA(2.0)']
# aggs = ["mean", "cov", "std", "0.005", "0.01", "0.025", "0.05", "0.1", "0.2", "0.5", "0.8", "0.9", "0.95", "0.975", "0.99", "0.995"]
aggs = ["mean", "0.1", "0.5", "0.9"]

In [ ]:
site_list = 'NZ_0_1_NB_1_1'
resample = 0.1
locations = []
grid = RegionGrid[site_list]
grid_locs = grid.load()

# remove empty location
l = grid_locs.index( (-34.7,172.7) )
grid_locs = grid_locs[0:l] + grid_locs[l+1:]

for gloc in grid_locs:
    loc = CodedLocation(*gloc, resolution=0.001)
    loc = loc.resample(float(resample)) if resample else loc
    locations.append(loc.resample(0.001).code)

In [ ]:
columns = ['lat', 'lon', 'vs30','imt', 'agg', 'level', 'hazard']
index = range(len(locations) * len(vs30_list) * len(imts) * len(aggs) * num_levels)
grid_summary_data = pd.DataFrame(columns=columns, index=index)

ind = 0
for i,res in enumerate(get_hazard_curves(locations, vs30_list, [hazard_id], imts, aggs)):
    lat = f'{res.lat:0.3f}'
    lon = f'{res.lon:0.3f}'
    for value in res.values:
        grid_summary_data.loc[ind,'lat'] = lat
        grid_summary_data.loc[ind,'lon'] = lon
        grid_summary_data.loc[ind,'vs30'] = res.vs30
        grid_summary_data.loc[ind,'imt'] = res.imt
        grid_summary_data.loc[ind,'agg'] = res.agg
        grid_summary_data.loc[ind,'level'] = value.lvl
        grid_summary_data.loc[ind,'hazard'] = value.val
        ind += 1

### compile data for either the short list ('points') or for the gridded sites ('grid')

In [6]:
data_type = 'points'

if data_type=='grid':
    site_label = '_grid-0pt1'
    summary_data = grid_summary_data
    vs30_list = list(np.unique(summary_data['vs30'].dropna()))
    
    site_list = 'NZ_0_1_NB_1_1'
    resample = 0.1
    locations = []
    grid = RegionGrid[site_list]
    grid_locs = grid.load()

    # remove empty location
    l = grid_locs.index( (-34.7,172.7) )
    grid_locs = grid_locs[0:l] + grid_locs[l+1:]

    for gloc in grid_locs:
        loc = CodedLocation(*gloc, resolution=0.001)
        loc = loc.resample(float(resample)) if resample else loc
        locations.append(loc.resample(0.001).code)
        
    sites = pd.DataFrame(dtype='str')
    for i,latlon in enumerate(locations):
        lat,lon = latlon.split('~')
        sites.loc[i,['lat','lon']] = [float(lat),float(lon)]
    sites['lat/lon'] = [(lat,lon) for lat,lon in zip(sites['lat'],sites['lon'])]

    
else:
    site_label=''
    summary_data = pts_summary_data
    vs30_list = list(np.unique(summary_data['vs30'].dropna()))
        
    site_list = sorted([loc['name'] for loc in custom_location_dict.values()])
    sites = pd.DataFrame(index=site_list,dtype='str')
    for loc in custom_location_dict.values():
        idx = (pts_summary_data['lat'].astype('float')==loc['latitude'])&(pts_summary_data['lon'].astype('float')==loc['longitude'])
        pts_summary_data.loc[idx,'name'] = loc['name']
        sites.loc[loc['name'],['lat','lon']] = [loc['latitude'],loc['longitude']]
    
        

In [7]:
imtl = list(np.unique(summary_data['level'].dropna()))
imtls = {}
for imt in imts:
    imtls[imt] = imtl

In [8]:
data = {}
data['metadata'] = {}
data['metadata']['quantiles'] = []
data['metadata']['acc_imtls'] = imtls
data['metadata']['sites'] = sites
data['metadata']['vs30s'] = vs30_list

In [9]:
idx_dict = {}

idx_dict['vs30'] = {}
for i_vs30,vs30 in enumerate(vs30_list):
    idx_dict['vs30'][vs30] = summary_data['vs30']==vs30
    
if data_type=='grid':
    idx_dict['lat'] = {}
    for i_lat,lat in enumerate(np.unique(sites['lat'])):
        idx_dict['lat'][lat] = summary_data['lat']==f'{float(lat):.3f}'

    idx_dict['lon'] = {}
    for i_lon,lon in enumerate(np.unique(sites['lon'])):
        idx_dict['lon'][lon] = summary_data['lon']==f'{float(lon):.3f}'
else:
    idx_dict['site'] = {}
    for i_site,site in enumerate(sites.index):
        idx_dict['site'][site] = summary_data['name']==site
    
idx_dict['imt'] = {}
for i_imt,imt in enumerate(imts):
    idx_dict['imt'][imt] = summary_data['imt']==imt

idx_dict['stat'] = {}
for i_stat,stat in enumerate(aggs):
    idx_dict['stat'][stat] = summary_data['agg']==stat 

In [10]:
hcurves = np.zeros([len(vs30_list),len(locations),len(imts),len(imtl),len(aggs)])

for i_vs30,vs30 in enumerate(vs30_list):
    print(f'Vs30: {vs30}')
    vs30_idx = idx_dict['vs30'][vs30]
    
    if data_type == 'grid':
        for i_latlon,latlon in enumerate(locations):
            print(f'\tLat/Lon: {latlon}')
            lat,lon = latlon.split('~')
            latlon_idx = (idx_dict['lat'][lat]) & (idx_dict['lon'][lon])

            for i_imt,imt in enumerate(imts):
                print(f'\t\tIMT: {imt}\t\t(Vs30: {vs30} at Lat/Lon: {latlon}  #{i_latlon+1} of {len(locations)})')
                imt_idx = idx_dict['imt'][imt]

                for i_stat,stat in enumerate(aggs):
                    print(f'\t\t\tStat: {stat}')
                    stat_idx = idx_dict['stat'][stat]

                    idx = vs30_idx & latlon_idx & imt_idx & stat_idx
                    hcurve = np.squeeze(summary_data[idx]['hazard'].to_numpy())

                    hcurves[i_vs30,i_latlon,i_imt,:,i_stat] = hcurve
                    
    else:
        for i_site,site in enumerate(sites.index):
            print(f'\tSite: {site}')
            site_idx = idx_dict['site'][site]

            for i_imt,imt in enumerate(imts):
                print(f'\t\tIMT: {imt}\t\t(Vs30: {vs30} at Site: {site}  #{i_site+1} of {len(sites)})')
                imt_idx = idx_dict['imt'][imt]

                for i_stat,stat in enumerate(aggs):
                    print(f'\t\t\tStat: {stat}')
                    stat_idx = idx_dict['stat'][stat]

                    idx = vs30_idx & site_idx & imt_idx & stat_idx
                    hcurve = np.squeeze(summary_data[idx]['hazard'].to_numpy())

                    hcurves[i_vs30,i_site,i_imt,:,i_stat] = hcurve

Vs30: 250
	Site: Auckland
		IMT: SA(0.5)		(Vs30: 250 at Site: Auckland  #1 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
		IMT: SA(1.5)		(Vs30: 250 at Site: Auckland  #1 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
	Site: Wellington
		IMT: SA(0.5)		(Vs30: 250 at Site: Wellington  #2 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
		IMT: SA(1.5)		(Vs30: 250 at Site: Wellington  #2 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
Vs30: 400
	Site: Auckland
		IMT: SA(0.5)		(Vs30: 400 at Site: Auckland  #1 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
		IMT: SA(1.5)		(Vs30: 400 at Site: Auckland  #1 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
	Site: Wellington
		IMT: SA(0.5)		(Vs30: 400 at Site: Wellington  #2 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9
		IMT: SA(1.5)		(Vs30: 400 at Site: Wellington  #2 of 2)
			Stat: mean
			Stat: 0.1
			Stat: 0.5
			Stat: 0.9


In [11]:
data['hcurves'] = {}
data['hcurves']['hcurves_stats'] = hcurves

### process data for the uniform hazard design intensities

In [12]:
data['metadata']['disp_imtls'] = convert_imtls_to_disp(imtls)
data['metadata']['quantiles'] = [float(q) for q in aggs[1:]]

# get poe values
print('Calculating PoE values.')
hazard_rps = np.array([25,50,100,250,500,1000,2500])
data['hazard_design'] = {}
data['hazard_design']['hazard_rps'] = hazard_rps.tolist()

for intensity_type in ['acc','disp']:
    data['hazard_design'][intensity_type] = {}
    data['hazard_design'][intensity_type]['stats_im_hazard'] = calculate_hazard_design_intensities(data,hazard_rps,intensity_type)


Calculating PoE values.


In [ ]:
if data_type=='grid':
    hf_name = f'v3_cmr-{cmr}_vs30-{vs30_list}_imt-{imts}{site_label}.hdf5'
else:
    hf_name = f'v3_cmr-{cmr}_vs30-all_R-factors.hdf5'
save_hdf(hf_name,data)

### process data for the risk informed design intensities

In [13]:
R_assumptions = {}

####
baseline_rp = 500
baseline_risk = 1.5e-5
cmr = 4
####

beta = 0.45
design_point = stats.lognorm(beta,scale=cmr).cdf(1)
p_fatality_given_collapse = 0.1

R_assumptions['baseline_risk'] = baseline_risk
R_assumptions['R_rps'] = [500,1000,2500]
for R_rp in R_assumptions['R_rps']:
    risk_factor = baseline_rp/R_rp
    risk_target = round(baseline_risk*risk_factor,12)
    R_assumptions[f'APoE: 1/{R_rp}'] = {'risk_factor': risk_factor,'risk_target': risk_target}

R_assumptions


risk_assumptions = {}
for R_rp in R_assumptions['R_rps']:
    key = f'APoE: 1/{R_rp}'
    risk_assumptions[key] = { 'risk_factor':R_assumptions[f'APoE: 1/{R_rp}']['risk_factor'],
                              'risk_target':R_assumptions[f'APoE: 1/{R_rp}']['risk_target'],
                              'R_rp':R_rp,
                              'collapse_risk_target': round(R_assumptions[f'APoE: 1/{R_rp}']['risk_target'] / p_fatality_given_collapse,12),
                              'cmr':cmr,
                              'beta':beta,
                              'design_point':design_point,
                              'p_fatality_given_collapse':p_fatality_given_collapse
                               }
    
risk_assumptions

{'baseline_risk': 1.5e-05,
 'R_rps': [500, 1000, 2500],
 'APoE: 1/500': {'risk_factor': 1.0, 'risk_target': 1.5e-05},
 'APoE: 1/1000': {'risk_factor': 0.5, 'risk_target': 7.5e-06},
 'APoE: 1/2500': {'risk_factor': 0.2, 'risk_target': 3e-06}}

{'APoE: 1/500': {'risk_factor': 1.0,
  'risk_target': 1.5e-05,
  'R_rp': 500,
  'collapse_risk_target': 0.00015,
  'cmr': 4,
  'beta': 0.45,
  'design_point': 0.001032732090754596,
  'p_fatality_given_collapse': 0.1},
 'APoE: 1/1000': {'risk_factor': 0.5,
  'risk_target': 7.5e-06,
  'R_rp': 1000,
  'collapse_risk_target': 7.5e-05,
  'cmr': 4,
  'beta': 0.45,
  'design_point': 0.001032732090754596,
  'p_fatality_given_collapse': 0.1},
 'APoE: 1/2500': {'risk_factor': 0.2,
  'risk_target': 3e-06,
  'R_rp': 2500,
  'collapse_risk_target': 3e-05,
  'cmr': 4,
  'beta': 0.45,
  'design_point': 0.001032732090754596,
  'p_fatality_given_collapse': 0.1}}

In [14]:
# get risk values
print('Calculating risk informed values.')
data['risk_design'] = {}
intensity_type = 'acc'
data['risk_design'][intensity_type] = {}
data['risk_design'][intensity_type]['risk_assumptions'] = risk_assumptions
data['risk_design'][intensity_type]['im_risk'] = calculate_risk_design_intensities(data,risk_assumptions)

Calculating risk informed values.
Processing Vs30: 250.
	Processing SA(0.5).
	Processing SA(1.5).

Processing Vs30: 400.
	Processing SA(0.5).
	Processing SA(1.5).



In [ ]:
update_hdf_for_risk(hf_name,data)